<a href="https://colab.research.google.com/github/poorya-abbasi/DL-HW04/blob/main/DL_HW04_HazmEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation**

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import pickle
from google.colab import drive 
print("Imported native libraries")

Imported native libraries


In [ ]:
#Config
model = 'resources/postagger.model'
chunkerModel = 'resources/chunker.model'
#External Libraries
!pip install hazm
!mkdir -p resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -o -qq resources/resources-0.5.zip -d resources
from __future__ import unicode_literals
from hazm import *
normalizer = Normalizer()
tagger = POSTagger(model=model)
chunker = Chunker(model=chunkerModel)
lemmatizer = Lemmatizer()
parser = DependencyParser(tagger=tagger, lemmatizer=lemmatizer)
#Log
print("Installed and imported external libraries")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 28.6 MB/s 
     |████████████████████████████████| 233 kB 40.4 MB/s 
     |████████████████████████████████| 1.4 MB 56.0 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=02259c73f1f09c3e1ada33bcdf44c7a6fd84edf0515b1bab02ca555c241477ae
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154524 sha256=6a25d816137afa5e2b27d2a32037995ec3f9ded77e00e110e3eec7cf8c785cbe
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
Installed and importe

In [ ]:
#Config
pathToDataset='/content/drive/MyDrive/PerSICK.csv'
pathToProcessedDataset = '/content/drive/MyDrive/PerSICKHazmSubjects.csv'
#Code
drive.mount('/content/drive')
dataset = pd.read_csv(pathToDataset)
dataset.insert(3, 'sentence1subjects', np.nan)
dataset.insert(4, 'sentence2subjects', np.nan)
dataset.insert(5, 'group', np.nan)
dataset['sentence1subjects'] = dataset['sentence1subjects'].astype(object)
dataset['sentence2subjects'] = dataset['sentence2subjects'].astype(object)
dataset['group'] = dataset['group'].astype(object)
print("Mounted to drive and read dataset with shape",dataset.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted to drive and read dataset with shape (3102, 6)


In [ ]:
testText = 'یک کودک خردسال در حال بالا رفتن از یک دیوار سنگ نوردی است که در داخل خانه قرار دارد'
testNormalizedText = normalizer.normalize(testText)
testSents = sent_tokenize(testNormalizedText)
result = ""
for sent in testSents:
  dg = parser.parse(word_tokenize(sent))
  for head, rel, dep in dg.triples():
    if rel == "SBJ":
      result = dep[0]
      break
print("Running an automated test to check if Subject detection is working correctly")
print("Detecting Subject for",testText)
expected = 'کودک'
print("Detected Subject",result,"which is",result==expected)
if(result==expected):
  print("Test passed!")
else:
  print("Test failed!")

Running an automated test to check if Subject detection is working correctly
Detecting Subject for یک کودک خردسال در حال بالا رفتن از یک دیوار سنگ نوردی است که در داخل خانه قرار دارد
Detected Subject کودک which is True
Test passed!


In [ ]:
dataset.head()

,score,sentence1,sentence2,sentence1subjects,sentence2subjects,group
0,4.5,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,گروهی از پسران در حیاط بازی می کنند و مردی در ...,NaN,NaN,NaN
1,3.2,گروهی از کودکان در خانه مشغول بازی هستند و هیچ...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,NaN,NaN,NaN
2,4.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,بچه ها در بیرون از خانه و در کنار یک مرد با لب...,NaN,NaN,NaN
3,3.4,بچه ها در بیرون از خانه و در کنار یک مرد با لب...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,NaN,NaN,NaN
4,3.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,NaN,NaN,NaN


In [ ]:
#Config
stopAtIndex = -1 #Set to -1 to never stop
#Function Defenition
def getSubjects(text):
  result = list()
  normalized = normalizer.normalize(text)
  sents = sent_tokenize(normalized)
  for sent in sents:
    dg = parser.parse(word_tokenize(sent))
    for head, rel, dep in dg.triples():
      if rel == "SBJ":
        result.append(dep[0])
  return result
#Code
if not os.path.exists(pathToProcessedDataset):
  for index, row in dataset.iterrows():
    if index == stopAtIndex : break
    try:    
      dataset.at[index,'sentence1subjects']=np.array(getSubjects(row['sentence1']))
      dataset.at[index,'sentence2subjects']=np.array(getSubjects(row['sentence2']))
      if index%10 == 0 :
        print("Processing done until index",index)
        dataset.to_csv('PerSICKHazmSubjects.csv',encoding='utf-8')
    except Exception as e:
      print("Parsing failed for index",index)
  print("Detecting subjects done")
else:
  dataset = pd.read_csv(pathToProcessedDataset)
  dataset.drop('Unnamed: 0', inplace=True, axis=1)
  dataset.insert(5, 'group', np.nan)
  dataset['group'] = dataset['group'].astype(object)

In [ ]:
dataset.head()

,score,sentence1,sentence2,sentence1subjects,sentence2subjects,group
0,4.5,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,گروهی از پسران در حیاط بازی می کنند و مردی در ...,['گروهی' 'پیرمردی'],['گروهی' 'مردی' 'زمینه'],NaN
1,3.2,گروهی از کودکان در خانه مشغول بازی هستند و هیچ...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,['گروهی' 'مردی'],['گروهی' 'پیرمردی'],NaN
2,4.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,بچه ها در بیرون از خانه و در کنار یک مرد با لب...,['پسران' 'مرد'],['بچه\u200cها'],NaN
3,3.4,بچه ها در بیرون از خانه و در کنار یک مرد با لب...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,['بچه\u200cها'],['گروهی' 'پیرمردی'],NaN
4,3.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,['پسران' 'مرد'],['گروهی' 'پیرمردی'],NaN


In [ ]:
def escape(text):
  converted = str(text)
  converted = re.sub("[^ا-ی]"," ",converted)
  converted = re.sub(' +', ' ',converted)
  return converted.strip()

def groupSubject(text):
  text = escape(text)
  maleGroup = ["پسر", "مرد"]
  femaleGroup = ["دختر", "زن"]
  childGroup = ["کودک", "بچه"]
  animalGroup = ["گربه", "سگ"]
  if any(ext in text for ext in maleGroup):
    return 0
  elif any(ext in text for ext in femaleGroup):
    return 1
  elif any(ext in text for ext in childGroup):
    return 2
  elif any(ext in text for ext in animalGroup):
    return 3
  else:
    return 4

def groupSubjects(one,two):
  if not any(item in escape(one).split(" ") for item in escape(two).split(" ")):
    return np.nan
  else:
    groupOne = groupSubject(one)
    groupTwo = groupSubject(two)
    if not groupOne == groupTwo:
      return 4
    else:
      return groupOne
for index, row in dataset.iterrows():
  sent1Subjects = row['sentence1subjects']
  sent2Subjects = row['sentence2subjects']
  group = groupSubjects(sent1Subjects,sent2Subjects)
  dataset.at[index,'group']=group
dataset.head()

,score,sentence1,sentence2,sentence1subjects,sentence2subjects,group
0,4.5,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,گروهی از پسران در حیاط بازی می کنند و مردی در ...,['گروهی' 'پیرمردی'],['گروهی' 'مردی' 'زمینه'],0
1,3.2,گروهی از کودکان در خانه مشغول بازی هستند و هیچ...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,['گروهی' 'مردی'],['گروهی' 'پیرمردی'],0
2,4.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,بچه ها در بیرون از خانه و در کنار یک مرد با لب...,['پسران' 'مرد'],['بچه\u200cها'],NaN
3,3.4,بچه ها در بیرون از خانه و در کنار یک مرد با لب...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,['بچه\u200cها'],['گروهی' 'پیرمردی'],NaN
4,3.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,['پسران' 'مرد'],['گروهی' 'پیرمردی'],NaN
